In [4]:
import pandas as pd
# Charger les données
df = pd.read_csv("sms_agressifs_dataset.csv")  # colonnes : 'text', 'label'

!pip install spacy
!python -m spacy download fr_core_news_sm


import spacy
nlp = spacy.load("fr_core_news_sm")

def nettoyer_spacy(texte):
    doc = nlp(texte.lower())
    tokens = [
        token.lemma_ for token in doc
        if not token.is_stop and not token.is_punct and token.is_alpha and len(token) > 1
    ]
    return " ".join(tokens)

df["clean_text"] = df["text"].astype(str).apply(nettoyer_spacy)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 86.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(df["clean_text"])

sequences = tokenizer.texts_to_sequences(df["clean_text"])
X = pad_sequences(sequences, maxlen=50, padding='post')

X_train, X_test, y_train, y_test = train_test_split(X, df["label"], test_size=0.2, random_state=42)



In [7]:
import tensorflow as tf
# Paramètres
max_words = 5000
max_len = 50
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=max_words, output_dim=64, input_length=max_len),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.build(input_shape=(None, max_len))
model.summary()

model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, batch_size=16)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 50, 64)         │       320,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 128)            │        66,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 390,209 (1.49 MB)

 Trainable params: 390,209 (1.49 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 7s 71ms/step - accuracy: 0.5763 - loss: 0.6740 - val_accuracy: 0.8750 - val_loss: 0.3524
Epoch 2/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 43ms/step - accuracy: 1.0000 - loss: 0.1174 - val_accuracy: 1.0000 - val_loss: 0.0016
Epoch 3/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 44ms/step - accuracy: 1.0000 - loss: 0.0012 - val_accuracy: 1.0000 - val_loss: 5.8533e-04
Epoch 4/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 44ms/step - accuracy: 1.0000 - loss: 3.5372e-04 - val_accuracy: 1.0000 - val_loss: 3.2324e-04
Epoch 5/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 76ms/step - accuracy: 1.0000 - loss: 1.9029e-04 - val_accuracy: 1.0000 - val_loss: 1.7071e-04


In [8]:
# Sauvegarder modèle au format recommandé
model.save("modele_lstm_sms.keras")

# Sauvegarder tokenizer
import pickle
with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)


In [9]:
import tensorflow as tf
import pickle
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 🔁 Charger modèle et tokenizer
model = tf.keras.models.load_model("modele_lstm_sms.keras")
with open("tokenizer.pkl", "rb") as f:
    tokenizer = pickle.load(f)

# ✉️ 10 nouveaux SMS à prédire
nouveaux_sms = [
    "Tu es une personne admirable, respectueuse et sincère.",
    "Tu n’apportes rien d’utile, comme toujours.",
    "Merci pour ton soutien bienveillant dans les moments difficiles.",
    "Arrête de jouer les victimes, tu fatigues tout le monde.",
    "Ta présence est rassurante et apaisante.",
    "T’es vraiment qu’un bon à rien, pathétique.",
    "J’apprécie ton honnêteté et ta gentillesse.",
    "On en a marre de ton comportement enfantin.",
    "Tu me motives à me dépasser chaque jour.",
    "Tais-toi un peu, t’es insupportable à écouter."
]

# 🧼 Prétraiter les textes comme dans l'entraînement
import spacy
nlp = spacy.load("fr_core_news_sm")

def nettoyer_spacy(text):
    doc = nlp(text.lower())
    tokens = [
        token.lemma_ for token in doc
        if not token.is_stop and not token.is_punct and token.is_alpha and len(token) > 1
    ]
    return " ".join(tokens)

nettoyés = [nettoyer_spacy(txt) for txt in nouveaux_sms]

# 🔢 Transformer et prédire
sequences = tokenizer.texts_to_sequences(nettoyés)
padded = pad_sequences(sequences, maxlen=50, padding='post')
probas = model.predict(padded)

# 💬 Afficher les prédictions
for i, sms in enumerate(nouveaux_sms):
    label = "Agressif 😠" if probas[i][0] >= 0.5 else "Non agressif 😊"
    print(f"{i+1:02d}. {sms} ➤ {label} (score: {probas[i][0]:.2f})")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 410ms/step
01. Tu es une personne admirable, respectueuse et sincère. ➤ Non agressif 😊 (score: 0.00)
02. Tu n’apportes rien d’utile, comme toujours. ➤ Non agressif 😊 (score: 0.44)
03. Merci pour ton soutien bienveillant dans les moments difficiles. ➤ Non agressif 😊 (score: 0.00)
04. Arrête de jouer les victimes, tu fatigues tout le monde. ➤ Agressif 😠 (score: 0.93)
05. Ta présence est rassurante et apaisante. ➤ Non agressif 😊 (score: 0.00)
06. T’es vraiment qu’un bon à rien, pathétique. ➤ Agressif 😠 (score: 1.00)
07. J’apprécie ton honnêteté et ta gentillesse. ➤ Non agressif 😊 (score: 0.00)
08. On en a marre de ton comportement enfantin. ➤ Agressif 😠 (score: 0.96)
09. Tu me motives à me dépasser chaque jour. ➤ Non agressif 😊 (score: 0.20)
10. Tais-toi un peu, t’es insupportable à écouter. ➤ Agressif 😠 (score: 0.91)
